In [1]:
import os

if (os.path.basename(os.getcwd()) == 'Notebooks'):
    os.chdir('..')

In [2]:
from wettbewerb import load_references
from preprocessing.preprocessing import *
from preprocessing.padding import *
from preprocessing.features import *
from utils.utils import *
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

from scipy.signal import find_peaks
import seaborn as sns
import math
import neurokit2 as nk
import scipy.io
import sklearn
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [3]:
ecg_leads, ecg_labels, fs, ecg_names = load_references(folder="data/training") # Importiere EKG-Dateien, zugehörige Diagnose, Sampling-Frequenz (Hz) und Name                                                # Sampling-Frequenz 300 Hz
train_data, train_labels, val_data, val_labels, test_data, test_labels = preprocess(ecg_leads, ecg_labels)

6000	 Dateien wurden geladen.


# Feature extraction
1. number of missing p peaks w.r.t r peaks
2. distribution score
3. sd1
4. sd2
5. sd1/sd2
6. beat rate (per sec for now)
7. dominant frequency

In [ ]:
X = []
y = []
for i in range(len(train_data)):
    sig = invert2(train_data[i])
    sig = nk.ecg_clean(sig, sampling_rate=300, method="neurokit")
    #sig = remove_noise_butterworth(sig, 300)
    r_peaks, p_peaks = find_peaks_r_p(sig, 300)
    
    len_r = len(r_peaks)
    len_p = len(p_peaks)
    score = distribution_score(r_peaks, sig)
    sd1, sd2, ratio = poincare_sd(r_peaks)
    dom_freq = dominant_freq(sig)
    beat_rate = beats_per_sec(sig, len_r)
    
    #feature array
    array = [len_r - len_p, score, sd1, sd2, ratio, beat_rate, dom_freq]
    
    X.append(array)
    y.append(train_labels[i])
X, y = np.array(X), np.array(y)    

C:\Users\Abhi\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Abhi\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\1_TUD\a_Summer 22\1_Dr. DeepAI\github\preprocessing\preprocessing.py:60: RuntimeWarning: Mean of empty slice.
  orig_peaks_mean = orig_sinal_peaks.mean()


In [24]:
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)

clf = RandomForestClassifier(max_depth = 30, random_state=0)
clf = clf.fit(X, y)